# Design of AI systems

## Assignment 7, Eric Johansson & Max Sonnelid

# Dialogue system

In [2]:
import numpy as np
from collections import defaultdict
import pandas as pd
import collections
import re
import random
from collections import Counter
from datetime import date
import calendar
from datetime import datetime

In [3]:
d = datetime.now()  
dayNumber = d.strftime("%d") 
currentDay = datetime.today().weekday()    

In [4]:
currentDay

0

## AllData stores all information. To add more information about a new field, just add a new method

In [5]:
class AllData():
    def weatherFacts(self, date, weekday, location):
        threshold = [0,0]
        if location == 'Gothenburg':
            threshold = [4,8]

        if location == 'Stockholm':
            threshold = [3,6]

        if location == 'New York':
            threshold = [4,5]
        
        if threshold == 'San Francisco':
            threshold = [1,2]

        rand = random.randint(0, 10)

        if  rand < threshold[0]:
            return 'rain' 
        
        elif  rand < threshold[1]:
            return 'cloudy'

        else: return 'sun'


    def foodFacts(self, meal, preference):
        if (meal == 'breakfast'):
            return ['Mahogny Coffee Bar', 'Gibraltargatan 4, 411 32 Göteborg'] ## potentially add opening hours
        
        if ((meal == 'lunch') or (meal == 'dinner')) and ((preference == 'thai') or (preference == 'sushi')):
            return ['Moon Thai Kitchen', 'Kristinelundsgatan 9, 411 37 Göteborg']

        if ((meal == 'lunch') or (meal == 'dinner')) and (preference == 'pizza'):
            return ['BOV', 'Gibraltargatan 20, 411 32 Göteborg']

        if ((meal == 'dinner')) and (preference == 'home cooking'):
            return ['Berzelius Bar & Matsal', 'Södra Vägen 20, 412 54 Göteborg']

        if ((meal == 'lunch')) and (preference == 'home cooking'):
            return ['Kåges Hörna', 'Stora Saluhallen, 411 17 Göteborg']

        else:
            return ['cook your own food', 'your own house']


    def transportFacts(self, startDest, endDest):
        if (startDest == 'Chalmers') and (endDest == 'Järntorget') or (startDest == 'Järntorget') and (endDest == 'Chalmers'):
            return 6, 5-datetime.now().minute%5 ## Departs every 5:th minute
        if (startDest == 'Vasaplatsen') and (endDest == 'Saltholmen') or (startDest == 'Saltholmen') and (endDest == 'Vasaplatsen'):
            return 11, 10-datetime.now().minute%10 ## Departs every 10:th minute
        if (startDest == 'Nordstan') and (endDest == 'Lindholmen') or (startDest == 'Lindholmen') and (endDest == 'Nordstan'):
            return 16, 15-datetime.now().minute%15 ## Departs every 15:th minute
        else:
            return 0, 0



## General_frame is the frame superclass

In [6]:
class General_frame():

    def __init__(self):
        self.necessary_questions = {}

    def updateVariables(self, string):
        return 0

    def add_question(self, topic, necessary):
        self.necessary_questions[topic] = ["", necessary, False]

    def ask_question(self, key):
        if (key == 'happy'):
            print("Are you happy with me?")
            var = str(input())
            res = re.search('yes|Yes|yeah|YEAH|absolutely|course|happy',var) ## Course is added if user types off course
            print('\n YOUR INPUT:', var, '\n')
            #self.updateVariables(var)
            if res:
                self.necessary_questions[key] = [res.group(), True, True]
                print('That is great to hear! Bye for now.')
            else:
                print('Ok, I am sorry to hear that. Then I guess I can not help you. You should ask Alexa, she might yeild some better answer...')
        

        if (key == 'moreHelp'):
            print('So, is there anything else I can help you with?')
            var = str(input())
            res = re.search('yes|Yes|yeah|YEAH|absolutely|course',var) ## Course is added if user types off course
            print('\n YOUR INPUT:', var, '\n')
            if res:
                #print('\n YOUR INPUT:', var, '\n')
                print('var = ', var)
                return var
            else:
                return None
            

    def satisfied(self):
        for key in self.necessary_questions:
            if (self.necessary_questions[key][2]==False):
                return False, key
            else:
                return True, key


## Subclass that handles questions regarding resturants

In [7]:
class Restaurant_frame(General_frame):

    def __init__(self):
        super(Restaurant_frame, self).__init__()
        self.add_question("meal", True)
        self.add_question("dish", True)  
        print('I see, you are hungry!')

    def updateVariables(self, string):
        dish = re.search('thai|pizza|burger|sushi|home cooking',string)
        if dish:
            self.necessary_questions["dish"] = [dish.group(), True, True]

        meal = re.search('breakfast|lunch|dinner',string)
        if meal:
            self.necessary_questions["meal"] = [meal.group(), True, True]

    def ask_question(self, key):
        super(Restaurant_frame, self).ask_question(key) 
                
        if (key == 'meal'):
            print("What type of meal do you want to eat? I mean like, do you want breakfast or anything else?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables(var)

        if (key == 'dish'):
            print("Are you looking for anything special?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables(var)

    def satisfied(self):
        for key in self.necessary_questions:
            if (self.necessary_questions[key][2]==False):
                return False, key
        else:
            return True, key

    def makeRecommendation(self):
        data = AllData()
        dish = self.necessary_questions['dish'][0]
        meal = self.necessary_questions['meal'][0]
        resturant, location = data.foodFacts(self.necessary_questions['meal'][0], self.necessary_questions['dish'][0])
        return f'Then i would reccomend you to go to {resturant}, which lies at {location}. They have great {dish} at {meal}-time!'

## Subclass that handles questions regarding weather

In [8]:


class Wheater_frame(General_frame):

    def __init__(self):
        super(Wheater_frame, self).__init__()
        self.add_question("weekday", True)
        self.add_question("date", True)
        self.add_question("location", True)
        self.necessary_questions["month"] = ['March', True, True]

        print('Ok, so you want to know more about the weather!')

    def dayToInt(self, str):
        if str=='Monday':
            return 0

        if str=='Tuesday':
            return 1

        if str=='Wednesday':
            return 2

        if str=='Thursday':
            return 3

        if str=='Friday':
            return 4
        
        if str=='Saturday':
            return 5

        if str=='Sunday':
            return 6

    def dayToStr(self, int):
        if int == 0:
            return 'Monday'

        if int == 1:
            return 'Tuesday'

        if int == 2:
            return 'Wednesday'

        if int == 3:
            return 'Thursday'

        if int == 4:
            return 'Friday'

        if int == 5:
            return 'Saturday'

        if int == 6:
            return 'Sunday'

    def getDate(self, string, next, weekDay=True):
        d = datetime.now()                              # Current day of the month
        dayNumber = int(d.strftime("%d"))                    #  Current day of the month
        currentDay = int(datetime.today().weekday())         # Number of weekday

        if (weekDay==False):
            if string=='tomorrow':
                return int(dayNumber) + 1, self.dayToStr(int(currentDay)+1)
            else:
                return int(dayNumber), self.dayToStr(int(currentDay))

        futureDay = int(self.dayToInt(string))                       # Number of wanted day
        
        if futureDay < currentDay:
            newNumber = 7 - currentDay + futureDay
        else:
            newNumber = futureDay-currentDay

        if next:
            date = 7 + dayNumber + newNumber
            return date
        else:
            return dayNumber + newNumber
       

    def updateVariables(self, string):
        

        day2 = re.search('Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday',string)
        if day2:
            date = self.getDate(day2.group(), next = False, weekDay = True)
            self.necessary_questions["weekday"] = [day2.group(), True, True]
            self.necessary_questions["date"] = [date, True, True]

        day = re.search('next (Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)',string) # try add | following
        if day:
            date = self.getDate(str(day.group(1)), next= True, weekDay = True)
            self.necessary_questions["weekday"] = [day.group(1), True, True]
            self.necessary_questions["date"] = [date, True, True]

        day3 = re.search('tomorrow|today|now|tonight', string)
        if day3:
            date, day = self.getDate(day3.group(), next = False, weekDay = False)
            self.necessary_questions["weekday"] = [day, True, True]
            self.necessary_questions["date"] = [date, True, True]

        location = re.search('Gothenburg|Stockholm|New York|San Francisco',string)
        if location:
            self.necessary_questions["location"] = [location.group(), True, True]



    def ask_question(self, key):
        super(Wheater_frame, self).ask_question(key) 
                
        if (key == 'location'):
            print("What location are you wondering about")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables(var)

        if (key == 'date'):
            print("Which day are you talking about?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables(var)

        if (key == 'weekday'):
            print("Which weekday are you talking about?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables(var)



    def satisfied(self):
        super(Wheater_frame, self).satisfied()
        for key in self.necessary_questions:
            if (self.necessary_questions[key][2]==False):
                return False, key
        else:
            return True, key

    def makeRecommendation(self):
        data = AllData()
        date = self.necessary_questions['date'][0]
        weekday = self.necessary_questions['weekday'][0]
        location = self.necessary_questions['location'][0]
        month = self.necessary_questions['month'][0]
        forecast = data.weatherFacts(date, weekday, location)
        return f'There will {forecast} in {location} on {weekday} the {date} of {month}'


## Subclass that handles questions regarding transport

In [9]:
class Transport_frame(General_frame):

    def __init__(self):
        super(Transport_frame, self).__init__()
        self.add_question("from", True)
        self.add_question("to", True)
        print('Ok, so you are going to places!')

    def updateVariables(self, string):
        startDest = re.search('from (Chalmers|Vasaplatsen|Nordstan|Järntorget|Saltholmen|Lindholmen)',string)
        if startDest:
            self.necessary_questions["from"] = [startDest.group(1), True, True]

        endDest = re.search('to (Chalmers|Vasaplatsen|Nordstan|Järntorget|Saltholmen|Lindholmen)',string) 
        if endDest:
            self.necessary_questions["to"] = [endDest.group(1), True, True]

    def ask_question(self, key):
        super(Transport_frame, self).ask_question(key) 
                
        if (key == 'from'):
            print("Where are coming from?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables('from ' + var)

        if (key == 'to'):
            print("And where are you going?")
            var = str(input())
            print('\n YOUR INPUT:', var, '\n')
            self.updateVariables('to ' + var)
    
    def satisfied(self):
        super(Transport_frame, self).satisfied()
        for key in self.necessary_questions:
            if (self.necessary_questions[key][2]==False):
                return False, key
        else:
            return True, key

    def makeRecommendation(self):
        data = AllData()
        startDest = self.necessary_questions['from'][0]
        endDest = self.necessary_questions['to'][0]
        answer, timeLeft = data.transportFacts(startDest, endDest)
        if answer == 0:
            return f'Ouh, it doesnt seem like there exists any buses or trams between these destinations. Instead, I booked a cab that will pick you up at {startDest} in five minutes and drive you to {endDest}. The tab is on me.' 

        return f'If you are going from {startDest} to {endDest}, then I recommend you to hop on line nr.{answer}. It leaves in {timeLeft} minutes'


## Virtural assistant is the class that handles request and generates proper frames

In [10]:
class VirtualAssistant():

    def __init__(self):
        self.frame = None
        print('Hi Sir/Madam! I am great at providing weather forecasts, finding nice restaurant or finding the next bus. If you want any help, just call at me and I will help you.')

    def generateFrame(self, topic):
        if (topic=='food'):
            return Restaurant_frame()
        if (topic == 'weather'):
            return Wheater_frame()
        if (topic == 'transport'):
            return Transport_frame()
        else:
            return General_frame()    
        

    def hey(self, answer=None):

        if answer == None:
            answer = str(input())
            print('\n YOUR INPUT:', answer, '\n')
            
        topic = self.find_topic(answer)
        
        while topic == 0:
            print('Could you further describe what you need help with?')
            answer = str(input())
            print('\n YOUR INPUT:', answer, '\n')
            topic = self.find_topic(answer)

        self.frame = self.generateFrame(topic)
        self.frame.updateVariables(answer)
        
        boolean = False
        while (boolean == False):
            boolean, key = self.frame.satisfied()
            if boolean == False:
                self.frame.ask_question(key)

        print(self.frame.makeRecommendation())
        #self.frame.ask_question('moreHelp')
        print('Is there anything else you want help with?')
        more = str(input())
        print('\n YOUR INPUT:', more, '\n')
        res = re.search('yes|Yes|yeah|YEAH|absolutely|course',more) ## Course is added if user types off course
        topic = self.find_topic(more)
        if res or (topic!=0):
            self.hey(answer=more)

        else:
            self.frame.ask_question('happy')    

    def find_topic(self, input):

        categories = ['weather', 'food', 'transport']
        weatherKeys = ['weather', 'rain', 'sun', 'umbrella', 'raincoat', 'outside', 'forecast']
        foodKeys = ['food', 'hungry', 'pizza', 'eat', 'starving', 'eaten', 'ate', 'restaurant', 'restaurants']
        transportKeys = ['transport', 'bus', 'travel', 'going', 'journey', 'tram', 'taxi', 'traffic', 'get']

        weather = 0
        food = 0
        transport = 0 
        
        sentence = input.split()
        counterWeather = Counter(weatherKeys)
        counterFood = Counter(foodKeys)
        counterTransp = Counter(transportKeys)

        for word in sentence:
            weather += counterWeather[word]
            food += counterFood[word]
            transport += counterTransp[word]
        
        result = categories[np.argmax([weather, food, transport])]
        
        if (weather > 0) or (food > 0) or (transport > 0):
            return result
        
        return 0    
        
        

# Creating our own Siri

In [11]:
siri = VirtualAssistant()

Hi Sir/Madam! I am great at providing weather forecasts, finding nice restaurant or finding the next bus. If you want any help, just call at me and I will help you.


## Getting help with multiple tasks

In [12]:
siri.hey()


 YOUR INPUT: Hey, i would like to eat pizza can you help me? 

I see, you are hungry!
What type of meal do you want to eat? I mean like, do you want breakfast or anything else?

 YOUR INPUT: Im thinking about dinner 

Then i would reccomend you to go to BOV, which lies at Gibraltargatan 20, 411 32 Göteborg. They have great pizza at dinner-time!
Is there anything else you want help with?

 YOUR INPUT: I also wonder if you know how to get to Saltholmen 

Ok, so you are going to places!
Where are coming from?

 YOUR INPUT: Vasaplatsen 

If you are going from Vasaplatsen to Saltholmen, then I recommend you to hop on line nr.11. It leaves in 4 minutes
Is there anything else you want help with?

 YOUR INPUT: yes one more thing 

Could you further describe what you need help with?

 YOUR INPUT: I would like to know what the weather in Stockholm is right now 

Ok, so you want to know more about the weather!
There will sun in Stockholm on Monday the 8 of March
Is there anything else you want h

## Asking about the weather

In [77]:
siri.hey()


 YOUR INPUT: I will soon be going to New York , do you think I need to bring my rain coat ? 

Ok, so you want to know more about the weather!
Which weekday are you talking about?

 YOUR INPUT: i will be leaving next Friday 

There will rain in New York on Friday the 12 of March
Is there anything else you want help with?

 YOUR INPUT: Can you check the weather tomorrow in Gothenburg ? 

Ok, so you want to know more about the weather!
There will cloudy in Gothenburg on Saturday the 6 of March
Is there anything else you want help with?

 YOUR INPUT: No thanks! 

Are you happy with me?

 YOUR INPUT: yeah 

That is great to hear! Bye for now.


## Getting help with transport

In [103]:
siri.hey()


 YOUR INPUT: I need to get to Lindholmen 

Ok, so you are going to places!
Where are coming from?

 YOUR INPUT: Chalmers is my current possition 

Ouh, it doesnt seem like there exists any buses or trams between these destinations. Instead, I booked a cab that will pick you up at Chalmers in five minutes and drive you to Lindholmen. The tab is on me.
Is there anything else you want help with?

 YOUR INPUT: Well, i also need to take the tram to Järntorget 

Ok, so you are going to places!
Where are coming from?

 YOUR INPUT: Chalmers still 

If you are going from Chalmers to Järntorget, then I recommend you to hop on line nr.6. It leaves in 5 minutes
Is there anything else you want help with?

 YOUR INPUT: No thanks! 

Are you happy with me?

 YOUR INPUT: Of course  

That is great to hear! Bye for now.
